<h1 align=center><font size = 5>Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)</font></h1>

## Install libraties

In [1]:
!pip install geopy #install geopy library if not present

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


## Import packages

In [2]:
import numpy as np # library to handle data in a vectorized manner

In [3]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [4]:
import json # library to handle JSON files

In [5]:
import requests # library to handle requests

In [6]:
#!conda install -c conda-forge geopy --yes # uncomment this line if this isn't installedb
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [7]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [9]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [10]:
import itertools #library for iterators, used for truth table generation

In [11]:
from math import radians, sin, cos, atan2, sqrt #for haversine

## User-defined Functions

In [12]:
def getNearbyVenues(df,names, lat, lng, radius):
    venues_list=[]
    for name in df:
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
         
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
#STILL NEEDS VERIFICATION
def getNearbyHotels(df,names, lat, lng, radius):
    hotels_list=[]
    for name in df:
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        rowsHotelsCSV = len(hotelsCSV)
        # return only relevant information for each nearby venue
        hotels_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_hotels = pd.DataFrame([item for hotel_list in hotels_list for item in hotel_list])
    nearby_hotels.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
def haversine(lat1,long1, lat2,long2):
    R = 6371 
    #print(R, e)
    theta1 = radians(lat1)
    theta2 = radians(lat2)
    deltatheta = radians(lat2 - lat1 )
    deltalambda = radians(long2 - long1 )
    a = sin(deltatheta/2) * sin(deltatheta/2) + cos(theta1) * cos(theta2) * sin(deltalambda/2) * sin(deltalambda/2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c 
    return d

## New York Data

#### Download NY dataset

In [15]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load data

In [16]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

#### Explore data

In [17]:
#comment out after: initial viewing of dataset, comment because too long!
#newyork_data

In [18]:
# Save all *features* keys into neighborhoods.  list
neighborhoods_data = newyork_data['features']

In [19]:
#comment out after: initial viewing of 'sliced' dataset, comment because too long!
#neighborhoods_data

#### Transform the data into a *pandas* dataframe

In [20]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [21]:
# loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [22]:
# preserve with Borough, Neighborhood, Latitude, Longitude
df_original = neighborhoods.copy()

In [23]:
#check number of neighborhoods per borough
df_original.groupby('Borough').count() 

,Neighborhood,Latitude,Longitude
Borough,,,
Bronx,52,52,52
Brooklyn,70,70,70
Manhattan,40,40,40
Queens,81,81,81
Staten Island,63,63,63


## Prepare New York city data and FourSquare API

### Use geopy library to get the latitude and longitude values 

In [24]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = 'ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555' # your Foursquare ID
CLIENT_SECRET = 'LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555
CLIENT_SECRET:LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX


In [26]:
# set variables: limit and radii of venues
LIMIT = 100 # limit of number of venues returned by Foursquare API, just a nice round num.

radius = 4024 # define radius in meters assuming 2.5 miles per hour with an average lower bound ofwalking for 20 a day assuming 8 hours

In [27]:
#FourSquare URL parameters
NY_Lat = '40.712776' #from https://www.latlong.net/convert-address-to-lat-long.html 
NY_Lon = '-74.005974' #from https://www.latlong.net/convert-address-to-lat-long.html 
NY_LL = NY_Lat + ',' + NY_Lon
Category = 'Hotel'#'Outdoor Sculpture'#Monument/Landmark'#''Hotel' 
Intent = 'global'
Near = 'New York, NY'
HotelID = '4bf58dd8d48988d1fa93173'

In [28]:
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&near={}&query={}&ll={}'.format(
    CLIENT_ID,
    CLIENT_SECRET, 
    VERSION, 
    Near,
    Category,
    NY_LL)

In [29]:
url

'https://api.foursquare.com/v2/venues/search?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&near=New York, NY&query=Hotel&ll=40.712776,-74.005974'

In [31]:
## request URL. remeber to comment out after to avoid quota issues
#results_hotels = requests.get(url).json()

In [32]:
# after verify json, comment out due to length
#results_hotels

In [33]:
#format json for loading
hotels_json = results_hotels['response'] ['venues'] 

In [34]:
#create hotels dataframe from json
hotels = json_normalize(hotels_json)

In [35]:
#get referralId as it changes with each request
referralId = hotels ['referralId'] [0] 

In [36]:
# eliminate venues inside hotel
hotels = hotels[hotels['referralId']==referralId]

In [37]:
# sort hotels according to name to verify
hotels.sort_values(by = ['name'], inplace = True)

In [38]:
# reindex hotels for readability
hotels.reset_index(inplace = True)

In [39]:
#inspect hotels 30 were returned
#hotels

In [40]:
# drop a few columns to make it more workable
hotels_d = hotels.drop(['categories', 'referralId', 'hasPerk',
                        'location.labeledLatLngs','location.distance','location.state','location.cc','location.city','location.country',
                       'venuePage.id'], 
                       axis=1)

#### manually eliminate invalid entries for NY hotel

In [41]:
#inside ace
hotels_d.drop_duplicates(subset=['location.lat','location.lng'],keep='first', inplace=True)

In [42]:
# drop ace lobby bar
indexNames = hotels_d[ hotels_d['name'] == 'Ace Hotel Lobby Bar' ].index

#hotels_d.drop(indexNames, inplace = True) #comment out once executed\

In [43]:
#drop dog hotel
indexNames = hotels_d[ hotels_d['name'] == 'New York Dog Spa & Hotel' ].index

#hotels_d.drop(indexNames, inplace = True) #comment out once executed

In [44]:
hotels_d.reset_index(inplace = True, drop = True)

In [45]:
hotels_d.shape
#able to remove the 2

(30, 9)

In [46]:
#save results once as a CSV to minimise FourSquare API calls
#hotels_d.to_csv('hotels.csv', index=False)

In [47]:
#read CSV
hotelsCSV = pd.read_csv('hotels.csv')

In [48]:
#verify CSV was read properly
#hotelsCSV

#### Visualize Hotels

In [49]:
# create map
map_hotels= folium.Map(location=[latitude, longitude], zoom_start=11)

In [50]:
hotelsCnt = len(hotelsCSV)

In [51]:
# set color scheme for the clusters
x = np.arange(hotelsCnt)
ys = [i + x + (i*x)**2 for i in range(hotelsCnt)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [52]:
# add markers to the map
markers_colors = []
for lat, lon, name in zip(hotels['location.lat'], hotelsCSV['location.lng'], hotelsCSV['name']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
         parse_html=False).add_to(map_hotels)  

In [53]:
map_hotels

## Cluster

#### Verify Boroughs

In [54]:
# comment out after: too many rows, 5 boroughs, 306 neighnorhoods and various Latitudes & Longitudes
#neighborhoods #.head()

#### Dedicated Neighborhoods DataFrame

In [55]:
# create a new dataframe
nhoods = pd.DataFrame()

hotelCSVCluster =pd.DataFrame()

In [56]:
nhoods['Neighborhood'] = neighborhoods['Neighborhood'] 

#

In [57]:
#make a copy for one-hot encoding
n_ohe = nhoods.copy()

hotelCSVClusterOHE = hotelCSVCluster.copy() 

#### k-means clustering

In [58]:
# set number of clusters aligned with number of borough
kclusters = 5

hotelClusters = 3

In [59]:
#lat,long only compatiblle with machine learning
cluster = neighborhoods.drop('Neighborhood', 1)
cluster.drop('Borough', 1, inplace = True)

#drp a lot for hotelsCSV
clusterHotels =pd.DataFrame()

In [60]:
# run k-means clustering for boroughs
kmeans_cluster = KMeans(n_clusters=kclusters, random_state=0).fit(cluster)

In [61]:
# run k-means clustering for hotels
#kmeans_clusterHotels = KMeans(n_clusters=hotelClusters, random_state=0).fit(clusterHotels)

In [62]:
# check cluster labels generated for each row in theboroughd dataframe
kmeans_cluster.labels_[0:4] 

array([4, 4, 4, 4], dtype=int32)

In [63]:
# check cluster labels generated for each row in the hotrlsCSVdataframe
#kmeans_clusterHotels.labels_[0:2] 

In [64]:
# add clustering labels
nhoods.insert(0, 'Cluster', kmeans_cluster.labels_)
#

In [65]:
#

In [66]:
#confirm cluster insertion
nhoods.head()

,Cluster,Neighborhood
0,4,Wakefield
1,4,Co-op City
2,4,Eastchester
3,4,Fieldston
4,4,Riverdale


#### Prepare for merge

In [67]:
#n,l,l
nhoods_latLong = df_original.copy()

nhoods_latLong.drop('Borough', 1, inplace = True)

In [68]:
merged = neighborhoods.copy()
merged_ohe = merged.copy

merged = merged.merge(nhoods , on='Neighborhood')

In [69]:
# ensure there are no duplicate neighborhoods
merged.drop_duplicates(subset='Neighborhood',keep='first', inplace=True)

merged.head()#shape

,Borough,Neighborhood,Latitude,Longitude,Cluster
0,Bronx,Wakefield,40.894705,-73.847201,4
1,Bronx,Co-op City,40.874294,-73.829939,4
2,Bronx,Eastchester,40.887556,-73.827806,4
3,Bronx,Fieldston,40.895437,-73.905643,4
4,Bronx,Riverdale,40.890834,-73.912585,4


#### Map clusters

In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [71]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [74]:
# add markers to the map
markers_colors = []
for lat, lon, nhood, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster']):
    label = folium.Popup(nhood + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [75]:
map_clusters

In [76]:
#we'll investgate further if there is a correspondence
df_BvC = df_original.copy()

In [77]:
df_original.head()#BvC

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [78]:
df_BvC.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [79]:
merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster
0,Bronx,Wakefield,40.894705,-73.847201,4
1,Bronx,Co-op City,40.874294,-73.829939,4
2,Bronx,Eastchester,40.887556,-73.827806,4
3,Bronx,Fieldston,40.895437,-73.905643,4
4,Bronx,Riverdale,40.890834,-73.912585,4


In [80]:
df_BvC = df_BvC.merge(merged, how='inner', on=['Neighborhood', 'Latitude','Longitude'])

In [81]:
df_BvC.head()

,Borough_x,Neighborhood,Latitude,Longitude,Borough_y,Cluster
0,Bronx,Wakefield,40.894705,-73.847201,Bronx,4
1,Bronx,Co-op City,40.874294,-73.829939,Bronx,4
2,Bronx,Eastchester,40.887556,-73.827806,Bronx,4
3,Bronx,Fieldston,40.895437,-73.905643,Bronx,4
4,Bronx,Riverdale,40.890834,-73.912585,Bronx,4


In [82]:
#Borough_x is fine since both coluns have similar values
df_BvC.groupby('Borough_x').count()

,Neighborhood,Latitude,Longitude,Borough_y,Cluster
Borough_x,,,,,
Bronx,52,52,52,52,52
Brooklyn,70,70,70,70,70
Manhattan,40,40,40,40,40
Queens,80,80,80,80,80
Staten Island,60,60,60,60,60


NOTE:  One-Hot Encoding the clusters seemed to add any further value as they coincided with NY city's 5 Boroughs (both visually and using groupby 

## Construct Dataframe for Analysis

#### Generate Truth Table (compatible with One-Hot Encoding)

This was omitted as merging took to long

In [83]:
# Set number of variables for truth table
numVar = 5 # All 10 POIs is too much computationally and for me to process in a short amount of time

In [84]:
#truth table genenrator funtion
table = list(itertools.product([0, 1], repeat=numVar))

In [85]:
#Comment out after: verify truth table construction
#table

#### Handle Points of Interest

In [86]:
# Points of Interest
POI_10 = ["Central Park",'Times Square','Grand Central Terminal','Broadway','Rockefeller Center']
POI_5= ["Central Park",'Times Square','Grand Central Terminal','Broadway','Rockefeller Center']
POI = POI_5        

##### Top 10 POI data 

https://en.wikipedia.org/wiki/Tourism_in_New_York_City#Most_visited_attractions

2018 65.2, 13.6 international

most are in Manhattan with the exception of Brooklyn
1. Central Park 42m 40.782222, -73.965278 geohack( https://tools.wmflabs.org/geohack/geohack.php?pagename=Tourism_in_New_York_City&params=40_46_56_N_73_57_55_W_&title=Central+Park) 40.782864, -73.965355 LatLong.net (https://www.latlong.net/convert-address-to-lat-long.html)
2.  Time Square 39.5 40.757, -73.986 40.759010, -73.984474
3.  Grand Central Terminal 21.6m 40.752813, -73.977215 40.752110, -73.977654
4. Theater District (incluging Broadway) 13m 40.759, -73.985  55.614700, 40.671299   Broadway 38.611641, -78.794952
5. Rockefeller Centre (including 30 Rock)12.8m 40.758611, -73.979167  40.068890, -83.126808
6.  Bryant Park 12m 40.754, -73.984 40.732690, -73.701640
7.  Prospect Park 10m 40.661667, -73.970833 40.619270,  -73.956960
8.  South Street Seaport 9m 40.706111, -74.003333	42.880280, -75.635620
9.  High Line 7.6m 40.747993, -74.004765 40.753440, -74.003890
10. Coney Island (including Boardwalk) 7.4m  40.574, -73.978, 44.588890 -75.138690 Boardwalk 40.576790, -73.943420
:
15. Ellis Island & Statue of Liberty 4.5m 40.694167, -74.043056 40.702150, -74.021460  Statue of Liberty40.689247, -74.044502

#### Construct Dataframe for POI

In [87]:
import pandas as pd

In [88]:
dict_POI = {'name':["Central Park",'Times Square','Grand Central Terminal','Broadway','Rockefeller Center'],
            'lat':[40.782,40.759,40.752,38.611,40.068],
             'lng': [-73.965,-73.984, -73.977,-78.794,-83.123]}

In [89]:
df_POI = pd.DataFrame()

In [90]:
df_POI = df_POI.from_dict(dict_POI)

In [91]:
df_POI.to_csv('POI.csv', index=False)

#### Visualize POI

In [92]:
# create map
map_pois= folium.Map(location=[latitude, longitude], zoom_start=11)

In [93]:
poisCnt = len(df_POI)

In [94]:
# set color scheme for the clusters
x = np.arange(poisCnt)
ys = [i + x + (i*x)**2 for i in range(poisCnt)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

In [95]:
# add markers to the map
markers_colors = []
for lat, lon, poi in zip(df_POI['lat'], df_POI['lng'], df_POI['name']):
    label = folium.Popup(poi, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
         parse_html=False).add_to(map_pois)  

In [96]:
map_pois

In [97]:
# Creating a dataframe object from list of tuples
TT = pd.DataFrame(table, columns = POI) 

In [98]:
#comment Out after: verify df with columns
#TT

#### Merge NY Hotels with Generated Truth Table

Removed because too computationally intensive from experience

In [99]:
rowsHotels_d = len(hotels_d)
colsHotels_d = len(hotels_d.columns)
#rowsTT = len(TT)
#colsTT = len(TT.columns)

In [100]:
iHotels_d = 0
cHotels_d = 0
listHotels_d =[]
strCol =''
listNew = []
listCols = []

iTT = 0
cTT= 0
intCol  = 0

In [101]:
while cHotels_d  < colsHotels_d:
    strCol = hotels_d.columns[cHotels_d]
    listCols.append(strCol ) 
       
    cHotels_d += 1 

In [102]:
listCols.append('haversine')

In [103]:
listCols.append('venues')

while cTT < colsTT:
    strCol = TT.columns[cTT]
    listCols.append(strCol ) 
       
    cTT += 1 

print(listCols)

#### this sets the entire row to NaN
####df_new.iloc[0,0] = np.na

In [104]:
df_new = pd.DataFrame(columns = listCols)

In [105]:
df_new

,index,id,name,location.address,location.crossStreet,location.lat,location.lng,location.postalCode,location.formattedAddress,haversine,venues


## Haversine

In [109]:
hotels_dm = hotels_d.copy()
#hotels_dm = hotels_d.insert('Haversine')

In [110]:
#add haversine column and set the entire thing to 0.0
hotels_dm['Haversine'] = 0.0

hotelsCSV['Haversine'] = 0.0 

In [111]:
#add venue column and set the entire thing to 0
hotels_dm['Venues'] = 0


hotelsCSV['Venues'] = 0

In [112]:
 #hotels_dm
hotelsCSV.head()

,index,id,name,location.address,location.crossStreet,location.lat,location.lng,location.postalCode,location.formattedAddress,Haversine,Venues
0,0,4a0e0f85f964a520bf751fe3,Ace Hotel New York,20 W 29th St,at Broadway,40.745858,-73.988121,10001,"['20 W 29th St (at Broadway)', 'New York, NY 1...",0.0,0
1,15,5bc65d8e1822230025fffcb1,Aliz Hotel,310 W 40th St,NaN,40.756207,-73.991576,10018,"['310 W 40th St', 'New York, NY 10018', 'Unite...",0.0,0
2,20,5370c510498e722315932c51,Archer Hotel New York,45 W 38th St,Between 5th And 6th Ave,40.751934,-73.984825,10018,"['45 W 38th St (Between 5th And 6th Ave)', 'Ne...",0.0,0
3,13,4ad46f88f964a520dde720e3,DoubleTree by Hilton Hotel New York City - Che...,128 W 29th St,at 7th Ave,40.746985,-73.991383,10001,"['128 W 29th St (at 7th Ave)', 'New York, NY 1...",0.0,0
4,3,48636489f964a520e0501fe3,Gramercy Park Hotel,2 Lexington Ave,at E 21st St,40.738533,-73.985651,10010,"['2 Lexington Ave (at E 21st St)', 'New York, ...",0.0,0


In [113]:
# test haversine function
haversineValue = haversine(hotelsCSV['location.lat'] [0], hotelsCSV ['location.lng'][0], df_POI['lat'] [0], df_POI['lng'] [0])

In [114]:
print (haversineValue)

4.465716968805757


In [115]:
hotelsCH = hotelsCSV.copy()

In [118]:
rowsHotelsCSV = len(hotelsCSV)
rowsPOI = len(df_POI)

#one hotel
iCoords = 0
haversineAcc = 0
print(rowsPOI)

while iCoords < rowsPOI:
    POILat = df_POI['lat'] [iCoords]
    POILng = df_POI['lat'] [iCoords] 
    POIName = df_POI['name'] [iCoords]
    print (iCoords,  hotelName, POIName, POILat, POILng )
    haversineAcc += haversine(hotelLat, hotelLng, POILat, POILng)
    iCoords += 1

In [119]:
#all hotels
iCoords = 0
iHotels = 0
haversineAcc = 0
print(rowsPOI)

while iHotels < rowsHotelsCSV:
    iCoords = 0
    haversineAcc = 0
    hotelName = hotelsCSV ['name'][iHotels]
    hotelLat = hotelsCSV ['location.lat'] [iHotels]
    hotelLng = hotelsCSV ['location.lng'] [iHotels]


    while iCoords < rowsPOI:
        POILat = df_POI['lat'] [iCoords]
        POILng = df_POI['lat'] [iCoords] 
        POIName = df_POI['name'] [iCoords]
        print (iCoords,  hotelName, POIName, POILat, POILng )
        haversineAcc += haversine(hotelLat, hotelLng, POILat, POILng)
        iCoords += 1
    
    hotelsCH.loc[iHotels, 'Haversine'] =haversineAcc
    iHotels += 1

5
0 Ace Hotel New York Central Park 40.782 40.782
1 Ace Hotel New York Times Square 40.759 40.759
2 Ace Hotel New York Grand Central Terminal 40.752 40.752
3 Ace Hotel New York Broadway 38.611 38.611
4 Ace Hotel New York Rockefeller Center 40.068 40.068
0 Aliz Hotel Central Park 40.782 40.782
1 Aliz Hotel Times Square 40.759 40.759
2 Aliz Hotel Grand Central Terminal 40.752 40.752
3 Aliz Hotel Broadway 38.611 38.611
4 Aliz Hotel Rockefeller Center 40.068 40.068
0 Archer Hotel New York Central Park 40.782 40.782
1 Archer Hotel New York Times Square 40.759 40.759
2 Archer Hotel New York Grand Central Terminal 40.752 40.752
3 Archer Hotel New York Broadway 38.611 38.611
4 Archer Hotel New York Rockefeller Center 40.068 40.068
0 DoubleTree by Hilton Hotel New York City - Chelsea Central Park 40.782 40.782
1 DoubleTree by Hilton Hotel New York City - Chelsea Times Square 40.759 40.759
2 DoubleTree by Hilton Hotel New York City - Chelsea Grand Central Terminal 40.752 40.752
3 DoubleTree by H

In [120]:
hotelsCH.to_csv('hotelsH.csv', index = False)

## Venues

In [122]:
hotelsCHV = hotelsCH.copy()

In [183]:
#test one
hotelLat = hotelsCHV['location.lat'][1]
hoteLng = hotelsCHV['location.lng'][1]

In [184]:
hotelLL = str(hotelLat)  +', '+ str(hotelLng)

In [185]:
print(hotelLL)

40.75620715052, -73.9966


In [186]:
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={}'.format(
    CLIENT_ID,
    CLIENT_SECRET, 
    VERSION, 
    hotelLL)

In [187]:
url

'https://api.foursquare.com/v2/venues/search?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&ll=40.75620715052, -73.9966'

In [188]:
 results_venues = requests.get(url).json()["response"] ['venues'] #['groups'][0]['items']

In [189]:
results_venues


[{'id': '514ca6f57043482a0c84e7ab',
  'name': 'Brooklyn Fare',
  'location': {'address': '431 W 37th St',
   'lat': 40.756094,
   'lng': -73.99659,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.756094,
     'lng': -73.99659}],
   'distance': 12,
   'postalCode': '10018',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['431 W 37th St',
    'New York, NY 10018',
    'United States']},
  'categories': [{'id': '52f2ab2ebcbc57f1066b8b46',
    'name': 'Supermarket',
    'pluralName': 'Supermarkets',
    'shortName': 'Supermarket',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
     'suffix': '.png'},
    'primary': True}],
  'venuePage': {'id': '51345309'},
  'referralId': 'v-1567349122',
  'hasPerk': False},
 {'id': '57fdb285498e7602cf010163',
  'name': '447 West 37th Street',
  'location': {'address': '447 W 37th St',
   'lat': 40.756374,
   'lng': -73.996864,
   'labeledLatLngs

In [190]:
#create hotels dataframe fromjson
venues = json_normalize(results_venues)

In [191]:
venues.shape

(30, 25)

In [181]:
hotelsCH.loc[1, 'Venues'] = len(venues)

In [171]:
hotelsCH

,index,id,name,location.address,location.crossStreet,location.lat,location.lng,location.postalCode,location.formattedAddress,Haversine,Venues
0,0,4a0e0f85f964a520bf751fe3,Ace Hotel New York,20 W 29th St,at Broadway,40.745858,-73.988121,10001,"['20 W 29th St (at Broadway)', 'New York, NY 1...",44133.723555,30
1,15,5bc65d8e1822230025fffcb1,Aliz Hotel,310 W 40th St,NaN,40.756207,-73.991576,10018,"['310 W 40th St', 'New York, NY 10018', 'Unite...",44130.697937,30
2,20,5370c510498e722315932c51,Archer Hotel New York,45 W 38th St,Between 5th And 6th Ave,40.751934,-73.984825,10018,"['45 W 38th St (Between 5th And 6th Ave)', 'Ne...",44130.357399,30
3,13,4ad46f88f964a520dde720e3,DoubleTree by Hilton Hotel New York City - Che...,128 W 29th St,at 7th Ave,40.746985,-73.991383,10001,"['128 W 29th St (at 7th Ave)', 'New York, NY 1...",44134.255495,30
4,3,48636489f964a520e0501fe3,Gramercy Park Hotel,2 Lexington Ave,at E 21st St,40.738533,-73.985651,10010,"['2 Lexington Ave (at E 21st St)', 'New York, ...",44135.857490,30
5,18,4b4bbe3ff964a52016a626e3,Hotel Edison,228 W 47th St,at 8th Ave,40.759666,-73.986088,10036,"['228 W 47th St (at 8th Ave)', 'New York, NY 1...",44127.703445,30
6,7,58d05f529465dd1d5f6500b3,Hotel Henri,37 W 24th St,NaN,40.743290,-73.991005,10010,"['37 W 24th St', 'New York, NY 10010', 'United...",44135.591353,30
7,12,54c0151d498e4c827296cd41,Hotel Indigo,171 Ludlow St,btw Houston & Stanton,40.721762,-73.988092,10002,"['171 Ludlow St (btw Houston & Stanton)', 'New...",44143.161038,30
8,2,4aaeb93ef964a5200d6320e3,Hotel Pennsylvania,401 7th Ave,at W 33rd St,40.749932,-73.990991,10001,"['401 7th Ave (at W 33rd St)', 'New York, NY 1...",44132.983517,30
9,24,4b061faff964a520ece822e3,Hotel Stanford,43 W 32nd St,at Broadway,40.748127,-73.987805,10001,"['43 W 32nd St (at Broadway)', 'New York, NY 1...",44132.739782,30


In [152]:
#loop for all hotels
iHotels = 0

#all hotels
iCoords = 0
iHotels = 0
haversineAcc = 0
print(rowsPOI)

while iHotels < rowsHotelsCSV:
    iCoords = 0
    haversineAcc = 0
    hotelName = hotelsCSV ['name'][iHotels]
    hotelLat = hotelsCSV ['location.lat'] [iHotels]
    hotelLng = hotelsCSV ['location.lng'] [iHotels]
    hotelLL = str(hotelLat)  +', '+ str(hotelLng)
   
    # create URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={}'.format(
        CLIENT_ID,
        CLIENT_SECRET, 
        VERSION, 
        hotelLL)
    print(iHotels,hotelLL, url)
    results_venues = requests.get(url).json()["response"] ['venues'] [iHotels]
    
    #create hotels dataframe fromjson
    venues = json_normalize(results_venues)
    venues.shape
    venueCnt = len(venues)
    
    hotelsCH.loc[iHotels, 'Venues'] = venueCnt
    
    iHotels += 1

5
0 40.74585807964396, -73.98812139191362 https://api.foursquare.com/v2/venues/search?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&ll=40.74585807964396, -73.98812139191362
1 40.75620715052, -73.99157595205548 https://api.foursquare.com/v2/venues/search?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&ll=40.75620715052, -73.99157595205548
2 40.751934399999996, -73.9848251 https://api.foursquare.com/v2/venues/search?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&ll=40.751934399999996, -73.9848251
3 40.7469851, -73.9913827 https://api.foursquare.com/v2/venues/search?&client_id=ISPOO44KLBPPIBVEULPPTWOPMH4AJB5KUVAGZEG23JFCB555&client_secret=LFOGHGUOYQZ2BOGSI0JXEOSNOMMFXLXU4BR5RQUPEWPOFVIX&v=20180605&ll=40.7469851, -73.9913827
4 40.738

In [193]:
hotelsCH.to_csv('hotelsHV.csv', index = False)

iHdm = 0
rHdm = len(hotels_dm)

while iHdm < rHdm:
    cnt = len(getNearbyVenues(hotels_dm,hotels_dm ['name'] [iHdm], hotels_dm['location.lat'] [iHdm], hotels_dm['location.lng'] [iHdm], radius ))
    print(iHdm.cnt)
    
    hotels_dm.loc[iHdm, 'Venues'] = cnt
    iHdm += 1

In [ ]:
hotels_dm['Venues']#head()
#why are some of the values 1100 or out of bounds when a limit was set?

iHotels_d = 0
iTT = 0
strDF =''
intDF = 0

while iHotels_d < rowsHotels_d:
    print("Hotels_d loop")
    listHotels_d = []
    cHotels_d = 0
    print('List Hotels_d - b4 append = ',listHotels_d)
    while cHotels_d < colsHotels_d: #for each col in hotel
        strCol = hotels_d.columns[cHotels_d]
        strDF = hotels_d[strCol][iHotels_d]
        print('in while hd, before loc',cHotels_d, strCol, iHotels_d,strDF)
        df_new.loc[iHotels_d, strCol] = strDF 
        print(cHotels_d,'List Hotels_d -after lov = ', strDF )
        cHotels_d += 1
        while iTT < rowsTT: #for wech row in TT
            print('while TT row')
            while cTT < colsTT: #for each col in TT
                strCol = TT.columns[cTT]
                intDF = hotels_d[intCol][iTT]
                print('in while ctt, before loc',cTT, intCol, iTT, intDF)
                df_new.loc[iTT, strCol] = intDF 
                print(cHotels_d,'List Hotels_d -after lov = ', intDF )
                cTT+= 1
                if cTT == colsTT:
                    cTT = 0
            iTT += 1
        
    
    iHotels_d += 1

df_new #.shape